<a href="https://colab.research.google.com/github/akkinapellisaicharan14/speechemotionrecognition/blob/main/speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os 
Root = "/content/drive/MyDrive"
os.chdir(Root)

In [6]:
ls


 12th.pdf
 500074200234_38713.jpg
'aadhar card.pdf'
'ABHA SAICHARAN'
 AKKI2002.pdf
 AKKI2004.pdf
'bank details.pdf'
'Colab Notebooks'/
 DOSE1.pdf
 DOSE2.pdf
'Food Recipe '/
 IMG_20210125_105627.jpg
 IMG_20210125_105634.jpg
 IMG_20210125_105642.jpg
 IMG_20210125_105647.jpg
 IMG_20210410_122743.jpg
 IMG_20220422_133002.jpg
 IMG_20220422_133014.jpg
 IMG-20220607-WA0030.jpg
 IMG-20221114-WA0002.jpg
 IMG-20230312-WA0015.jpg
 IMG-20230312-WA0016.jpg
 IMG_20230325_154120.jpg
 KALYANI.pdf
 pan.jpeg
 Screenshot_2022-03-05-12-45-54-691_com.android.chrome.jpg
 speech-emotion-recognition-ravdess-data/
 SUDHARSHAN.pdf
 TC.pdf
'Vit photos'/
'WhatsApp Image 2020-11-18 at 10.02.38 (2).jpeg'
'Xll Mark sheet.pdf'


In [51]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score




In [64]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [12]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [49]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data//Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)
   
 

In [65]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [74]:
x_train

array([[-4.95576782e+02,  1.16502113e+01, -2.18401623e+01, ...,
         2.36127945e-03,  1.51480746e-03,  1.02561142e-03],
       [-7.36479919e+02,  3.56320229e+01, -4.69407558e+00, ...,
         3.28449678e-05,  2.40549271e-05,  1.38549231e-05],
       [-7.18260376e+02,  3.15717583e+01,  3.41409135e+00, ...,
         4.00241697e-05,  3.25012552e-05,  1.94277363e-05],
       ...,
       [-4.35057098e+02, -1.31232154e+00, -3.15596752e+01, ...,
         4.74787783e-03,  2.95279035e-03,  1.27994292e-03],
       [-5.55144531e+02,  3.69400215e+01, -1.88234501e+01, ...,
         5.85853631e-05,  7.47469167e-05,  6.13598531e-05],
       [-5.51100586e+02,  4.01095428e+01, -1.60702705e+01, ...,
         5.49309421e-04,  5.55153820e-04,  3.91397509e-04]])

In [66]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(22, 8)


In [67]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [68]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [69]:
#Train the model
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:605: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [70]:
#Predict for the test set
y_pred=model.predict(x_test)

In [73]:
y_pred

array(['calm', 'calm', 'happy', 'calm', 'calm', 'calm', 'fearful', 'calm'],
      dtype='<U7')

In [71]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 37.50%
